In [1]:
# The code was removed by Watson Studio for sharing.

In [2]:
# 百度坐标系(BD-09)转WGS84坐标系
import math
x_pi = 3.14159265358979324 * 3000.0 / 180.0
pi = 3.1415926535897932384626 # π
a = 6378245.0 
ee = 0.00669342162296594323 # 
def bd09_to_gcj02(bd_lon, bd_lat):
    x = bd_lon - 0.0065
    y = bd_lat - 0.006
    z = math.sqrt(x * x + y * y) - 0.00002 * math.sin(y * x_pi)
    theta = math.atan2(y, x) - 0.000003 * math.cos(x * x_pi)
    gg_lng = z * math.cos(theta)
    gg_lat = z * math.sin(theta)
    return [gg_lng, gg_lat]

def gcj02_to_wgs84(gc_lon, gc_lat):
    dlat = _transformlat(gc_lon-105, gc_lat-35)
    dlon = _transformlng(gc_lon-105, gc_lat-35)
    radlat = gc_lat/180.0*pi
    magic = math.sin(radlat)
    magic = 1 -ee*magic*magic
    sqrtmagic = math.sqrt(magic)
    dlat = (dlat*180.0)/((a*(1-ee))/(magic*sqrtmagic)*pi)
    dlon = (dlon*180.0)/(a/sqrtmagic * math.cos(radlat)*pi)
    mglat = gc_lat +dlat
    mglon = gc_lon +dlon
    return [gc_lon*2 - mglon, gc_lat*2 - mglat]

def bd09_to_wgs84(bd_lon, bd_lat):
    lon, lat = bd09_to_gcj02(bd_lon, bd_lat)
    return gcj02_to_wgs84(lon, lat)

def _transformlat(lng, lat):
    ret = -100.0 + 2.0 * lng + 3.0 * lat + 0.2 * lat * lat + \
     0.1 * lng * lat + 0.2 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
      math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lat * pi) + 40.0 *
      math.sin(lat / 3.0 * pi)) * 2.0 / 3.0
    ret += (160.0 * math.sin(lat / 12.0 * pi) + 320 *
      math.sin(lat * pi / 30.0)) * 2.0 / 3.0
    return ret
 
def _transformlng(lng, lat):
    ret = 300.0 + lng + 2.0 * lat + 0.1 * lng * lng + \
     0.1 * lng * lat + 0.1 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
      math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lng * pi) + 40.0 *
      math.sin(lng / 3.0 * pi)) * 2.0 / 3.0
    ret += (150.0 * math.sin(lng / 12.0 * pi) + 300.0 *
      math.sin(lng / 30.0 * pi)) * 2.0 / 3.0
    return ret

In [3]:
import json
from urllib.request import urlopen,quote
import requests, csv
import pandas as pd
def get_coordinates(api_key, address, verbose = False):
    url = 'http://api.map.baidu.com/geocoder/v2/?address='
    output = 'json'
    ak = api_key
    address = quote(address) # some address is in Chinese
    uri = url + address+'&output='+output+'&ak='+ak
    req = urlopen(uri)
    res = req.read().decode() # to unicode
    temp = json.loads(res)
    lat_bd = temp['result']['location']['lat']
    lon_bd = temp['result']['location']['lng']
    lon, lat = bd09_to_wgs84(lon_bd, lat_bd)
    return [lat, lon]
address = '北京天安门广场, China'
beijing_center = get_coordinates(api_key, address)

print('Cooridnate of {}:{}'.format(address, beijing_center))

Cooridnate of 北京天安门广场, China:[39.902323140192586, 116.39153806571855]


In [4]:
!pip install shapely
!pip install pyproj

In [5]:
import shapely.geometry
import pyproj
import math
def lonlat_to_xy(lon, lat):
    proj_lonlat = pyproj.Proj(proj='latlong', datum ='WGS84')
    proj_xy = pyproj.Proj(proj='utm', zone = 33, datum = 'WGS84')
    xy = pyproj.transform(proj_lonlat, proj_xy, lon, lat)
    return xy[0], xy[1]
def xy_to_lonlat(x,y):
    proj_lonlat = pyproj.Proj(proj = 'latlong', datum = 'WGS84')
    proj_xy = pyproj.Proj(proj = 'utm', zone =33, datum = 'WGS84')
    lonlat = pyproj.transform(proj_xy, proj_lonlat, x, y)
    return lonlat[0], lonlat[1]
def calc_xy_distance(x1,y1,x2,y2):
    dx=x2-x1
    dy=y2-y1
    return math.sqrt(dx*dx + dy*dy)

print('Beijing center longitude ={},latitude ={}'.format(beijing_center[1],beijing_center[0]))
x,y = lonlat_to_xy(beijing_center[1], beijing_center[0])
print('Beijing center UTM X ={}, Y={}'.format(x,y))
lo, la = xy_to_lonlat(x,y)
print('Beijing center longtitude ={}, latitude={}'.format(lo, la))

Beijing center longitude =116.39153806571855,latitude =39.902323140192586
Beijing center UTM X =6736246.766234557, Y=11475272.15261937
Beijing center longtitude =116.39153806571892, latitude=39.90232314019351


In [6]:
#create a hexagonal grid of cells: we offset every other row, and adjust vertical row spacing so that every cell center is equally distant from all it's neighbors.
beijing_center_x, beijing_center_y = lonlat_to_xy(beijing_center[1],beijing_center[0])

k = math.sqrt(3)/2 # Vertical offset for hexagonal grid cells
x_min = beijing_center_x -6000
x_step = 600
y_min = beijing_center_y - 6000 - (int(21/k)*k*600-12000)/2
y_step = 600*k

latitudes =[]
longitudes =[]
distances_from_center = []
xs=[]
ys=[]
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2 == 0 else 0
    for j in range(0, 21):
        x=x_min+j*x_step+x_offset
        distance_from_center = calc_xy_distance(beijing_center_x, beijing_center_y, x, y)
        if(distance_from_center<=6001):
            lon, lat = xy_to_lonlat(x,y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)
print(len(latitudes), 'candidate neighborhood centers generated.')

364 candidate neighborhood centers generated.


In [7]:
!pip install folium
import folium

In [8]:
map_beijing = folium.Map(location = beijing_center, zoom_start = 12)
folium.Marker(beijing_center, popup = 'Bejing_center').add_to(map_beijing)
for lat, lon in zip(latitudes, longitudes):
    folium.Circle([lat,lon], radius = 300, color='blue').add_to(map_beijing)
map_beijing

In [9]:
import requests
#address = '39.83637707633588,115.58006911450369'
address=str(beijing_center[0])+','+str(beijing_center[1])
url = 'http://api.map.baidu.com/geocoder?output=json&key=f247cdb592eb43ebac6ccd27f796e2d2&location=' + str(address)
response = requests.get(url)
answer = response.json()
print(answer)

{'status': 'OK', 'result': {'location': {'lng': 116.391538, 'lat': 39.902323}, 'formatted_address': '北京市西城区厂甸甲9', 'business': '和平门,前门,宣武门', 'addressComponent': {'city': '北京市', 'direction': '附近', 'distance': '21', 'district': '西城区', 'province': '北京市', 'street': '厂甸', 'street_number': '甲9'}, 'cityCode': 131}}


In [10]:
import pandas as pd
def get_address(api_key, latitude, longitude, verbose = False):
        ak = api_key
        address = str(latitude)+','+str(longitude)
        url = 'http://api.map.baidu.com/geocoder?output=json&key='+ak+'&location=' + str(address)
        response = requests.get(url).json()
        result = response['result']
        address = result['formatted_address']
        return address

addr = get_address(api_key, beijing_center[0],beijing_center[1])
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(beijing_center[0],beijing_center[1],addr))




Reverse geocoding check
-----------------------
Address of [39.902323140192586, 116.39153806571855] is: 北京市西城区厂甸甲9


In [12]:
print('Obtaining location addresses:', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(api_key, lat, lon)
    if address is None:
        address = 'NO ADDRESS'
    addresses.append(address)
    print('.', end='')
print('done')

Obtaining location addresses:............................................................................................................................................................................................................................................................................................................................................................................done


In [13]:
addresses[160:170]

['北京市西城区北新华街94号',
 '北京市西城区前门西大街97号',
 '北京市西城区香炉营东巷2号院-1号楼',
 '北京市西城区东椿树胡同甲20号',
 '北京市西城区裘家街66号',
 '北京市西城区包头章胡同23号',
 '北京市西城区南横东街147号',
 '北京市西城区儒福里50号',
 '北京市西城区自新路40号',
 '北京市西城区里仁街6号']

In [14]:
import pandas as pd
df_locations = pd.DataFrame({'Address':addresses,
                            'Latitude': latitudes,
                            'Longitude': longitudes,
                            'X':xs,
                            'Y':ys,
                            'Distance from center':distance_from_center})
df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center
0,北京市西城区三里河东路甲8号院-2门-地下室-209室,39.922680,116.353506,6.734447e+06,1.146956e+07,8286.73639
1,北京市西城区二七剧场路9,39.919270,116.352128,6.735047e+06,1.146956e+07,8286.73639
2,北京市西城区三里河东路32号,39.915860,116.350750,6.735647e+06,1.146956e+07,8286.73639
3,北京市西城区白云路1号604-A室,39.912450,116.349371,6.736247e+06,1.146956e+07,8286.73639
4,北京市西城区白云观街北里,39.909041,116.347994,6.736847e+06,1.146956e+07,8286.73639
5,北京市西城区白云路,39.905632,116.346616,6.737447e+06,1.146956e+07,8286.73639
6,北京市西城区莲花池东路24号院,39.902223,116.345238,6.738047e+06,1.146956e+07,8286.73639
7,北京市西城区南礼士路5-3/5-2号,39.926876,116.359411,6.733547e+06,1.147008e+07,8286.73639
8,北京市西城区月坛西街乙2号院-4号楼,39.923465,116.358032,6.734147e+06,1.147008e+07,8286.73639
9,北京市西城区月坛西街24号院-1号楼,39.920055,116.356652,6.734747e+06,1.147008e+07,8286.73639


In [15]:
df_locations.to_pickle('./locations.pkl')

In [16]:
# The code was removed by Watson Studio for sharing.

In [17]:
food_category = '4d4b7105d754a06374d81259'
japanese_restaurant_categories =['4bf58dd8d48988d111941735','55a59bace4b013909087cb0c','55a59bace4b013909087cb30',
                                 '55a59bace4b013909087cb21','55a59bace4b013909087cb06','55a59bace4b013909087cb1b',
                                 '55a59bace4b013909087cb1e','55a59bace4b013909087cb18','55a59bace4b013909087cb24',
                                 '55a59bace4b013909087cb15','55a59bace4b013909087cb27','55a59bace4b013909087cb12',
                                 '4bf58dd8d48988d1d2941735','55a59bace4b013909087cb2d','55a59a31e4b013909087cb00',
                                 '55a59af1e4b013909087cb03','55a59bace4b013909087cb2a','55a59bace4b013909087cb0f',
                                 '55a59bace4b013909087cb33','55a59bace4b013909087cb09','55a59bace4b013909087cb36']
def is_restaurant(categories, specific_filter = None):
    restaurant_words = ['restaurant','dinner','taverna','steakhouse']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return[(cat['name'],cat['id']) for cat in categories]

def format_address(location):
    address = ','.join(location['formattedAddress'])
    address = address.replace(',中国', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius = 500, limit =100):
    version = '20180924'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
    client_id, client_secret, version, lat, lon, category, radius, limit)
    results = requests.get(url).json()['response']['groups'][0]['items']
    venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    return venues

In [18]:

import pickle

def get_restaurants(lats, lons):
    restaurants = {}
    japanese_restaurants = {}
    location_restaurants = []
    
    print('Obtaining venues and candidate locations:', end = '')
    for lat, lon in zip(lats, lons):
        venues = get_venues_near_location(lat,lon,food_category,foursquare_client_id, foursquare_client_secret, radius = 350, limit =100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_japanese = is_restaurant(venue_categories, specific_filter=japanese_restaurant_categories)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1],venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_japanese, x, y)
                if venue_distance<=300:
                    area_restaurants.append(restaurant)
                restaurants[venue_id]=restaurant
                if is_japanese:
                    japanese_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print('.', end='')
    print('done.')
    return restaurants, japanese_restaurants, location_restaurants

#Try to load local file system in case we did this before
restaurants = []
japanese_restaurants = {}
location_restaurants = []
loaded = False
try:
    with open('restaurants_350.pkl','rb') as f:
        restaurants = pickle.load(f)
    with open('japanese_restaurant_350.pkl','rb') as f:
        japanese_restaurants = pickle.load(f)
    with open('location_restaurants_350,pkl','rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass

if not loaded:
    restaurants, japanese_restaurants, location_restaurants = get_restaurants(latitudes, longitudes)
    with open('restaurants_350.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('japanese_restaurants_350.pkl','wb') as f:
        pickle.dump(japanese_restaurants, f)
    with open('location_restaurants_350.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)

Obtaining venues and candidate locations:............................................................................................................................................................................................................................................................................................................................................................................done.


In [19]:
import numpy as np

print('Total number of restaurants:', len(restaurants))
print('Total number of Japanese restaurants:', len(japanese_restaurants))
print('Percentage of Japanese restaurants:{:.2f}%'.format(len(japanese_restaurants)/len(restaurants)*100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 729
Total number of Japanese restaurants: 42
Percentage of Japanese restaurants:5.76%
Average number of restaurants in neighborhood: 3.17032967032967


In [20]:
print('List of all restaurants')
print('------------------------')
for r in list(restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(restaurants))

List of all restaurants
------------------------
('58e1d846126ae87997a6c0fe', '渔芙南', 39.923274970665744, 116.35229147504235, '北京市,北京市', 123, False, 6734398.005945203, 11469376.618661549)
('4e70a942a8098087a0313a93', '苏浙汇', 39.91900435194665, 116.35195134237236, '中国', 33, False, 6735096.135895262, 11469547.837947581)
('4d72e84b4ab5224b43bec697', 'Jin Ding Xuan (金鼎轩)', 39.91818251039085, 116.35243601067891, '北京西城区金融大街甲9号D1楼(近中国银监会)\xa0\xa0,北京市,北京市', 123, False, 6735209.338586989, 11469648.742942836)
('4fb8dad6e4b0d0dd7e9081f9', '新荣记 (金融街)', 39.918044516501126, 116.35238371432716, '西城区金融大街11号国际酒店B1,北京市,北京市', 138, False, 6735233.486208684, 11469649.174042083)
('4ccff55548b837046002efde', 'Steak Exchange Restaurant + Bar', 39.917705136967896, 116.35049856686783, 'Intercontinental Beijing Level 5,北京市,北京市', 206, False, 6735360.411057179, 11469433.257130904)
('4b058791f964a520789a22e3', 'Prego', 39.9161838599768, 116.35334443421237, '9B Financial Street,北京市,北京市, 100140', 331, False, 6735495.02

In [21]:
print('List of Japanese restaurants')
print('---------------------------')
for r in list(japanese_restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(japanese_restaurants))

List of Japanese restaurants
---------------------------
('51975cb4498ef767e213326d', 'Hatsune (隐泉之语)', 39.914308333103286, 116.35417660836649, '2 Jinchengfang St (at 4/F of Financial Street Shopping Centre),北京市,北京市', 121, True, 6735763.902083648, 11470057.245145997)
('507be851e4b0da23869fd9fa', '吉野家', 39.909900465082906, 116.35191098761673, '百盛M层', 347, True, 6736558.218595962, 11469997.510832766)
('4b680bb7f964a5201a652be3', '福神日本拉面居酒料理', 39.917277396082, 116.35871355031134, '西城區金城坊街3號-6號,北京市,北京市', 94, True, 6735113.088806983, 11470469.350271685)
('4bc694e0b387d13ac52dc2e8', '道乐日式菜馆 金融街店', 39.91212997893622, 116.35707057619179, '通泰大厦,北京市,北京市', 169, True, 6736002.033095341, 11470523.582602363)
('4dbb951193a08f9274a919b5', '喜多屋', 39.898530826874634, 116.35388836264238, '北京市,北京市', 203, True, 6738306.445159415, 11470809.938952155)
('4cdfcc823644a093568d549f', '面爱面 长椿街店', 39.897926363070766, 116.35698410355705, '宣武区长椿街甲7号,北京市,北京市, 100053', 313, True, 6738284.255709484, 11471222.772993118)

In [22]:
print('Restaurants around location')
print('---------------------------')
for i in range(100, 110):
    rs = location_restaurants[i][:8]
    names = ', '.join([r[1] for r in rs])
    print('Restaurants around location {}: {}'.format(i+1, names))

Restaurants around location
---------------------------
Restaurants around location 101: 聚德华天
Restaurants around location 102: 
Restaurants around location 103: Haidilaohuoguo
Restaurants around location 104: 
Restaurants around location 105: gyu-kaku 牛角日式烧肉
Restaurants around location 106: 六部口涮肉 Liubukou Meat in Hotpot, China Kitchen Grand Mercure BeiJing, 来涮蜀串串香
Restaurants around location 107: 
Restaurants around location 108: 黄记煌三汁焖锅宣武门店, 呷哺呷哺 (庄胜崇光店)
Restaurants around location 109: 西堤牛排 北京庄胜崇光店, 禾绿回转寿司, 紫荆豪庭港式粤菜
Restaurants around location 110: 英光私房菜


Let's now see all the collected restaurants in our area of interest on map, and let's also show Italian restaurants in different color.

In [23]:
map_beijing = folium.Map(location=beijing_center, zoom_start=13)
folium.Marker(beijing_center, popup='天安门').add_to(map_beijing)
for res in restaurants.values():
    lat = res[2]; lon = res[3]
    is_japanese = res[6]
    color = 'red' if is_japanese else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_beijing)
map_beijing

## Analaysis ##

Count the number of restaurants in every area candidate:

In [24]:
location_restaurants_count = [len(res) for res in location_restaurants]

df_locations['Restaurants in area'] = location_restaurants_count

print('Average number of restaurants in every area with radius=300m:', np.array(location_restaurants_count).mean())

df_locations.head(10)

Average number of restaurants in every area with radius=300m: 3.17032967032967


,Address,Latitude,Longitude,X,Y,Distance from center,Restaurants in area
0,北京市西城区三里河东路甲8号院-2门-地下室-209室,39.922680,116.353506,6.734447e+06,1.146956e+07,8286.73639,1
1,北京市西城区二七剧场路9,39.919270,116.352128,6.735047e+06,1.146956e+07,8286.73639,4
2,北京市西城区三里河东路32号,39.915860,116.350750,6.735647e+06,1.146956e+07,8286.73639,4
3,北京市西城区白云路1号604-A室,39.912450,116.349371,6.736247e+06,1.146956e+07,8286.73639,2
4,北京市西城区白云观街北里,39.909041,116.347994,6.736847e+06,1.146956e+07,8286.73639,3
5,北京市西城区白云路,39.905632,116.346616,6.737447e+06,1.146956e+07,8286.73639,2
6,北京市西城区莲花池东路24号院,39.902223,116.345238,6.738047e+06,1.146956e+07,8286.73639,3
7,北京市西城区南礼士路5-3/5-2号,39.926876,116.359411,6.733547e+06,1.147008e+07,8286.73639,4
8,北京市西城区月坛西街乙2号院-4号楼,39.923465,116.358032,6.734147e+06,1.147008e+07,8286.73639,4
9,北京市西城区月坛西街24号院-1号楼,39.920055,116.356652,6.734747e+06,1.147008e+07,8286.73639,1


calculate the distance to nearest Japanese restaurant from every area candidate center

In [25]:
distances_to_japanese_restaurant = []

for area_x, area_y in zip(xs, ys):
    min_distance = 10000
    for res in japanese_restaurants.values():
        res_x = res[7]
        res_y = res[8]
        d = calc_xy_distance(area_x, area_y, res_x, res_y)
        if d<min_distance:
            min_distance = d
    distances_to_japanese_restaurant.append(min_distance)

df_locations['Distance to Japanese restaurant'] = distances_to_japanese_restaurant
df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center,Restaurants in area,Distance to Japanese restaurant
0,北京市西城区三里河东路甲8号院-2门-地下室-209室,39.922680,116.353506,6.734447e+06,1.146956e+07,8286.73639,1,1130.261669
1,北京市西城区二七剧场路9,39.919270,116.352128,6.735047e+06,1.146956e+07,8286.73639,4,874.725533
2,北京市西城区三里河东路32号,39.915860,116.350750,6.735647e+06,1.146956e+07,8286.73639,4,514.375095
3,北京市西城区白云路1号604-A室,39.912450,116.349371,6.736247e+06,1.146956e+07,8286.73639,2,539.995013
4,北京市西城区白云观街北里,39.909041,116.347994,6.736847e+06,1.146956e+07,8286.73639,3,527.116477
5,北京市西城区白云路,39.905632,116.346616,6.737447e+06,1.146956e+07,8286.73639,2,992.022654
6,北京市西城区莲花池东路24号院,39.902223,116.345238,6.738047e+06,1.146956e+07,8286.73639,3,1280.168258
7,北京市西城区南礼士路5-3/5-2号,39.926876,116.359411,6.733547e+06,1.147008e+07,8286.73639,4,1614.958415
8,北京市西城区月坛西街乙2号院-4号楼,39.923465,116.358032,6.734147e+06,1.147008e+07,8286.73639,4,1043.313757
9,北京市西城区月坛西街24号院-1号楼,39.920055,116.356652,6.734747e+06,1.147008e+07,8286.73639,1,537.509543


In [26]:
print('Average distance to closest Japanese restaurant from each area center:', df_locations['Distance to Japanese restaurant'].mean())

Average distance to closest Japanese restaurant from each area center: 1158.2072949390931


Let's create a map showing heatmap / density of restaurants and try to extract some meaningfull info from that. 

In [27]:
beijing_boroughs_url = 'https://raw.githubusercontent.com/echarts-maps/echarts-china-cities-js/master/geojson/shape-with-internal-borders/beijing.geojson'
beijing_boroughs = requests.get(beijing_boroughs_url).json()


def boroughs_style(feature):
    return { 'color': 'blue', 'fill': False }

In [28]:
restaurant_latlons = [[res[2], res[3]] for res in restaurants.values()]

japanese_latlons = [[res[2], res[3]] for res in japanese_restaurants.values()]

In [29]:
from folium import plugins
from folium.plugins import HeatMap

map_beijing = folium.Map(location=beijing_center, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_beijing) #cartodbpositron cartodbdark_matter
HeatMap(restaurant_latlons).add_to(map_beijing)
folium.Marker(beijing_center).add_to(map_beijing)
folium.Circle(beijing_center, radius=1000, fill=False, color='white').add_to(map_beijing)
folium.Circle(beijing_center, radius=2000, fill=False, color='white').add_to(map_beijing)
folium.Circle(beijing_center, radius=3000, fill=False, color='white').add_to(map_beijing)
folium.GeoJson(beijing_boroughs, style_function=boroughs_style, name='geojson').add_to(map_beijing)
map_beijing

In [30]:
#heatmap for Japanese restaurant only
map_beijing = folium.Map(location=beijing_center, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_beijing) #cartodbpositron cartodbdark_matter
HeatMap(japanese_latlons).add_to(map_beijing)
folium.Marker(beijing_center).add_to(map_beijing)
folium.Circle(beijing_center, radius=1000, fill=False, color='white').add_to(map_beijing)
folium.Circle(beijing_center, radius=2000, fill=False, color='white').add_to(map_beijing)
folium.Circle(beijing_center, radius=3000, fill=False, color='white').add_to(map_beijing)
folium.GeoJson(beijing_boroughs, style_function=boroughs_style, name='geojson').add_to(map_beijing)
map_beijing

In [31]:
roi_x_min = beijing_center_x - 3000
roi_y_max = beijing_center_y 
roi_width = 5000
roi_height = 5000
roi_center_x = roi_x_min + 2500
roi_center_y = roi_y_max - 2500
roi_center_lon, roi_center_lat = xy_to_lonlat(roi_center_x, roi_center_y)
roi_center = [roi_center_lat, roi_center_lon]

map_berlin = folium.Map(location=roi_center, zoom_start=14)
HeatMap(restaurant_latlons).add_to(map_beijing)
folium.Marker(beijing_center).add_to(map_beijing)
folium.Circle(roi_center, radius=2500, color='white', fill=True, fill_opacity=0.4).add_to(map_beijing)
folium.GeoJson(beijing_boroughs, style_function=boroughs_style, name='geojson').add_to(map_beijing)
map_beijing

In [32]:
k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_step = 100
y_step = 100 * k 
roi_y_min = roi_center_y - 2500

roi_latitudes = []
roi_longitudes = []
roi_xs = []
roi_ys = []
for i in range(0, int(51/k)):
    y = roi_y_min + i * y_step
    x_offset = 50 if i%2==0 else 0
    for j in range(0, 51):
        x = roi_x_min + j * x_step + x_offset
        d = calc_xy_distance(roi_center_x, roi_center_y, x, y)
        if (d <= 2501):
            lon, lat = xy_to_lonlat(x, y)
            roi_latitudes.append(lat)
            roi_longitudes.append(lon)
            roi_xs.append(x)
            roi_ys.append(y)

print(len(roi_latitudes), 'candidate neighborhood centers generated.')

2261 candidate neighborhood centers generated.


In [ ]:
def count_restaurants_nearby(x, y, restaurants, radius=250):    
    count = 0
    for res in restaurants.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=radius:
            count += 1
    return count

def find_nearest_restaurant(x, y, restaurants):
    d_min = 100000
    for res in restaurants.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=d_min:
            d_min = d
    return d_min

roi_restaurant_counts = []
roi_japanese_distances = []

print('Generating data on location candidates... ', end='')
for x, y in zip(roi_xs, roi_ys):
    count = count_restaurants_nearby(x, y, restaurants, radius=250)
    roi_restaurant_counts.append(count)
    distance = find_nearest_restaurant(x, y, japanese_restaurants)
    roi_japanese_distances.append(distance)
print('done.')

In [ ]:
# Let's put this into dataframe
df_roi_locations = pd.DataFrame({'Latitude':roi_latitudes,
                                 'Longitude':roi_longitudes,
                                 'X':roi_xs,
                                 'Y':roi_ys,
                                 'Restaurants nearby':roi_restaurant_counts,
                                 'Distance to Japanese restaurant':roi_japanese_distances})

df_roi_locations.head(10)


OK. Let us now filter those locations: we're interested only in locations with no more than two restaurants in radius of 250 meters, and no Janpanese restaurants in radius of 400 meters.

In [ ]:
good_res_count = np.array((df_roi_locations['Restaurants nearby']<=2))
print('Locations with no more than two restaurants nearby:', good_res_count.sum())

good_ita_distance = np.array(df_roi_locations['Distance to Japanese restaurant']>=400)
print('Locations with no Japanese restaurants within 400m:', good_ita_distance.sum())

good_locations = np.logical_and(good_res_count, good_ita_distance)
print('Locations with both conditions met:', good_locations.sum())

df_good_locations = df_roi_locations[good_locations]


In [ ]:
good_latitudes = df_good_locations['Latitude'].values
good_longitudes = df_good_locations['Longitude'].values

good_locations = [[lat, lon] for lat, lon in zip(good_latitudes, good_longitudes)]

map_beijing = folium.Map(location=roi_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_beijing)
HeatMap(restaurant_latlons).add_to(map_beijing)
folium.Circle(roi_center, radius=2500, color='white', fill=True, fill_opacity=0.6).add_to(map_beijing)
folium.Marker(beijing_center).add_to(map_beijing)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_beijing) 
folium.GeoJson(beijing_boroughs, style_function=boroughs_style, name='geojson').add_to(map_beijing)
map_beijing

In [ ]:
#Let's now show those good locations in a form of heatmap:
map_beijing = folium.Map(location=roi_center, zoom_start=14)
HeatMap(good_locations, radius=25).add_to(map_beijing)
folium.Marker(beijing_center).add_to(map_beijing)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_beijing)
folium.GeoJson(beijing_boroughs, style_function=boroughs_style, name='geojson').add_to(map_beijing)
map_beijing

Let us now cluster those locations to create centers of zones containing good locations. Those zones, their centers and addresses will be the final result of our analysis. 

In [ ]:
from sklearn.cluster import KMeans

number_of_clusters = 15

good_xys = df_good_locations[['X', 'Y']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(good_xys)

cluster_centers = [xy_to_lonlat(cc[0], cc[1]) for cc in kmeans.cluster_centers_]

map_beijing = folium.Map(location=roi_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_beijing)
HeatMap(restaurant_latlons).add_to(map_beijing)
folium.Circle(roi_center, radius=2500, color='white', fill=True, fill_opacity=0.4).add_to(map_beijing)
folium.Marker(beijing_center).add_to(map_beijing)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='green', fill=True, fill_opacity=0.25).add_to(map_beijing) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_beijing)
folium.GeoJson(beijing_boroughs, style_function=boroughs_style, name='geojson').add_to(map_beijing)
map_beijing

In [ ]:
#city map without heatmap
map_beijing = folium.Map(location=roi_center, zoom_start=14)
folium.Marker(beijing_center).add_to(map_beijing)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=250, color='#00000000', fill=True, fill_color='#0066ff', fill_opacity=0.07).add_to(map_beijing)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_beijing)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='green', fill=False).add_to(map_beijing) 
folium.GeoJson(beijing_boroughs, style_function=boroughs_style, name='geojson').add_to(map_beijing)
map_beijing

Finaly, let's reverse geocode those candidate area centers to get the addresses which can be presented to stakeholders.

In [ ]:
candidate_area_addresses = []
print('==============================================================')
print('Addresses of centers of areas recommended for further analysis')
print('==============================================================\n')
for lon, lat in cluster_centers:
    addr = get_address(api_key, lat, lon)
    candidate_area_addresses.append(addr)    
    x, y = lonlat_to_xy(lon, lat)
    d = calc_xy_distance(x, y, beijing_center_x, beijing_center_y)
    print('{}{} => {:.1f}km from 天安门'.format(addr, ' '*(50-len(addr)), d/1000))
    